Size of training set: **10683 records**

Size of test set: **2671 records**

#### FEATURES:

Airline: The name of the airline.

Date_of_Journey: The date of the journey

Source: The source from which the service begins.

Destination: The destination where the service ends.

Route: The route taken by the flight to reach the destination.

Dep_Time: The time when the journey starts from the source.

Arrival_Time: Time of arrival at the destination.

Duration: Total duration of the flight.

Total_Stops: Total stops between the source and destination.

Additional_Info: Additional information about the flight

Price: The price of the ticket

In [1]:
# Imports
# !pip install xlrd
# !pip install openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder # OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_excel('data/Flight_Ticket_Participant_Datasets/Data_Train.xlsx')
test = pd.read_excel('data/Flight_Ticket_Participant_Datasets/Test_set.xlsx')

train.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


#### To do:

- [x] Look at the value counts for levels in each of the categorical features
- [ ] Encode `Airline`
- [x] `Date_of_Journey` to be datetime
- [ ] Encode `Source` and `Destination`
- [x] Split `Route` and find the number of stops (rename to `Total_Stops` and drop the original ?)
- [ ] Calculate time delta from `Duration` (perhaps, drop `Dep_Time` and `Arrival_Time`)
- [ ] Encode `Additional_Info`

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Date_of_Journey  2671 non-null   object
 2   Source           2671 non-null   object
 3   Destination      2671 non-null   object
 4   Route            2671 non-null   object
 5   Dep_Time         2671 non-null   object
 6   Arrival_Time     2671 non-null   object
 7   Duration         2671 non-null   object
 8   Total_Stops      2671 non-null   object
 9   Additional_Info  2671 non-null   object
dtypes: object(10)
memory usage: 208.8+ KB


In [5]:
# drop missing value obs.
train.dropna(inplace=True)

In [6]:
print(train.shape, test.shape)

(10682, 11) (2671, 10)


In [7]:
combined_dataframe = pd.concat([train.drop(['Price'], axis=1), test], ignore_index=True)
combined_dataframe.shape

(13353, 10)

In [8]:
combined_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13353 entries, 0 to 13352
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          13353 non-null  object
 1   Date_of_Journey  13353 non-null  object
 2   Source           13353 non-null  object
 3   Destination      13353 non-null  object
 4   Route            13353 non-null  object
 5   Dep_Time         13353 non-null  object
 6   Arrival_Time     13353 non-null  object
 7   Duration         13353 non-null  object
 8   Total_Stops      13353 non-null  object
 9   Additional_Info  13353 non-null  object
dtypes: object(10)
memory usage: 1.0+ MB


In [9]:
train_index = range(0, train.shape[0])
test_index = range(train.shape[0], combined_dataframe.shape[0])

price = train['Price']


# print(len(train_index), len(price.values.tolist()))

In [10]:
combined_dataframe['DOJ'] = pd.to_datetime(combined_dataframe['Date_of_Journey'], infer_datetime_format=True)

In [11]:
combined_dataframe.head(4)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,DOJ
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,2019-03-24
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,2019-05-01
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,2019-06-09
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,2019-05-12


In [12]:
combined_dataframe.drop(['Date_of_Journey'], axis=1, inplace=True)
combined_dataframe.rename(columns={"DOJ":"Date_of_Journey"}, inplace=True)

combined_dataframe.head(3)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Date_of_Journey
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,2019-03-24
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,2019-05-01
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,2019-06-09


In [13]:
# OPTION 2: day names instead of dayofweek, and use One Hot.

combined_dataframe['Day_of_Journey'] = combined_dataframe.Date_of_Journey.dt.dayofweek

In [14]:
# Create a new feature `num_stops` from the Route ; by counting the number of stops as seen in the route.
# `Route` and `Total_Stops` may be dropped.

combined_dataframe['Total_Stops_num'] = combined_dataframe['Route'].str.split(' → ', expand=False)
combined_dataframe['Total_Stops_num'] = combined_dataframe['Total_Stops_num'].apply(lambda x: len(x) if isinstance(x, list) else np.nan)

combined_dataframe.head(4)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Date_of_Journey,Day_of_Journey,Total_Stops_num
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,2019-03-24,6,2
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,2019-05-01,2,4
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,2019-06-09,6,4
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,2019-05-12,6,3


In [15]:
# `Duration` can be coerced to time in hours.
# `Arrival_Time` and `Dep_Time` may be dropped.

combined_dataframe['Duration_hours'] = combined_dataframe['Duration'].str.split('h', expand=True)[0]

In [16]:
# Little bit of clean up is required for the engineered feature here
combined_dataframe['Duration_hours'] = combined_dataframe['Duration_hours'].str.replace('[a-zA-Z]','').astype('int64')

In [17]:
# Using reindex instead of .loc, because this let's you handle NaNs - although we don't have any right now.

train = combined_dataframe.reindex(train_index)
test = combined_dataframe.reindex(test_index)
test.reset_index(inplace=True, drop=True)

train['Price'] = price.values

print(train.shape, test.shape)

train.head(4)

(10682, 14) (2671, 13)


,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Date_of_Journey,Day_of_Journey,Total_Stops_num,Duration_hours,Price
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,2019-03-24,6,2,2,3897
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,2019-05-01,2,4,7,7662
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,2019-06-09,6,4,19,13882
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,2019-05-12,6,3,5,6218


In [18]:
test.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Date_of_Journey,Day_of_Journey,Total_Stops_num,Duration_hours
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info,2019-06-06,3,3,10
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info,2019-05-12,6,3,4
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included,2019-05-21,1,3,23
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info,2019-05-21,1,3,13
4,Air Asia,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info,2019-06-24,0,2,2


In [19]:
# Check if train date range is overlapping with test date range

print("Training date range: {} - {}\t Testing date range: {} - {}".format(
    train['Date_of_Journey'].min(), train['Date_of_Journey'].max(), 
    test['Date_of_Journey'].min(), test['Date_of_Journey'].max())
     )

Training date range: 2019-03-01 00:00:00 - 2019-06-27 00:00:00	 Testing date range: 2019-03-01 00:00:00 - 2019-06-27 00:00:00


#### Observation

- The date range of training set and test set are overlapping (infact, spans the same range!)
    - The problem could be approached as a regression rather than a timeseries
    - Convert the datetime to day of week and encode the levels

#### Value counts

In [20]:
# \033[1m - ANSI for Bold characters
# \033[0m - ANSI to reset formatting for following characters

for col in train.select_dtypes(include=['object']).columns:
    print("\n\033[1m" + col + "\033[0m\n")
    print(train[col].value_counts())


Airline

Jet Airways                          3849
IndiGo                               2053
Air India                            1751
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

Source

Delhi       4536
Kolkata     2871
Banglore    2197
Mumbai       697
Chennai      381
Name: Source, dtype: int64

Destination

Cochin       4536
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kolkata       381
Name: Destination, dtype: int64

Route

DEL → BOM → COK                2376
BLR → DEL                      1552
CCU → BOM → BLR                 979
CCU → BLR                       724
BOM → HYD                      

In [21]:
for col in train.select_dtypes(include=['object']).columns:
    print("\n\033[1m" + col + "\033[0m\n")
    print(list(set(train[col].unique()) - set(test[col].unique())))


Airline

['Trujet']

Source

[]

Destination

[]

Route

['BOM → BLR → CCU → BBI → HYD', 'BOM → DED → DEL → HYD', 'BOM → CCU → HYD', 'CCU → IXZ → MAA → BLR', 'BLR → LKO → DEL', 'CCU → GAU → IMF → DEL → BLR', 'BOM → BDQ → DEL → HYD', 'BOM → BHO → DEL → HYD', 'BOM → NDC → HYD', 'BLR → HBX → BOM → AMD → DEL', 'BOM → MAA → HYD', 'BLR → TRV → COK → DEL', 'BOM → JDH → JAI → DEL → HYD', 'BOM → COK → MAA → HYD', 'CCU → IXA → BLR', 'BLR → CCU → BBI → HYD → DEL', 'CCU → BOM → PNQ → BLR', 'BLR → HBX → BOM → NAG → DEL', 'BLR → HBX → BOM → BHO → DEL', 'CCU → VTZ → BLR', 'BOM → JLR → HYD', 'BOM → RPR → VTZ → HYD', 'BOM → IDR → DEL → HYD', 'CCU → IXB → DEL → BLR', 'BLR → PNQ → DEL', 'CCU → RPR → HYD → BLR', 'BOM → UDR → DEL → HYD', 'BOM → VNS → DEL → HYD', 'DEL → BBI → COK', 'BOM → BBI → HYD', 'BLR → CCU → BBI → DEL', 'BLR → IDR → DEL']

Dep_Time

['13:40', '21:35', '21:55', '22:25', '12:15', '03:05', '12:05', '21:40', '04:15', '00:20', '12:45', '10:05', '12:20', '01:40', '06:35', '11:05', '15:25', 

In [22]:
# Among the categorical attributes, only `Airline` and `Additional_Info` are of use
# To verify above code manually.

for col in ['Airline', 'Additional_Info']:
    print("\n\033[1m" + col + "\033[0m\n")
    print(train[col].unique().tolist())
    print("\n")
    print(test[col].unique().tolist())


Airline

['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet', 'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia', 'Vistara Premium economy', 'Jet Airways Business', 'Multiple carriers Premium economy', 'Trujet']


['Jet Airways', 'IndiGo', 'Multiple carriers', 'Air Asia', 'Air India', 'Vistara', 'SpiceJet', 'Vistara Premium economy', 'GoAir', 'Multiple carriers Premium economy', 'Jet Airways Business']

Additional_Info

['No info', 'In-flight meal not included', 'No check-in baggage included', '1 Short layover', 'No Info', '1 Long layover', 'Change airports', 'Business class', 'Red-eye flight', '2 Long layover']


['No info', 'In-flight meal not included', 'No check-in baggage included', '1 Long layover', 'Business class', 'Change airports']


#### Observations

- `Airline`: Trujet is present only in train and not in the test set
- `Additional_Info`: Few of the levels are not present in the test set

Therefore, **OneHotEncoding** doesn't seem to be a good fit here.

- Below attributes,
    - `Day_of_Journey`, `Source`, `Destination` may be **OneHot** encoded (manually verified the unique occurences in train/test)

In [23]:
X = train.drop(['Price'], axis=1).copy()
y = train['Price'].copy()

In [24]:
print(X.shape, y.shape)

(10682, 13) (10682,)


In [25]:
X_test = test.copy()

In [26]:
# Drop irrelevant attributes

X.drop(['Route', 'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops', 'Date_of_Journey'], inplace=True, axis=1)
X.rename({"Total_Stops_num":"Total_Stops", "Duration_hours":"Duration"}, inplace=True)

X.head()

,Airline,Source,Destination,Additional_Info,Day_of_Journey,Total_Stops_num,Duration_hours
0,IndiGo,Banglore,New Delhi,No info,6,2,2
1,Air India,Kolkata,Banglore,No info,2,4,7
2,Jet Airways,Delhi,Cochin,No info,6,4,19
3,IndiGo,Kolkata,Banglore,No info,6,3,5
4,IndiGo,Banglore,New Delhi,No info,4,3,4


In [27]:
X_test.drop(['Route', 'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops', 'Date_of_Journey'], inplace=True, axis=1)
X_test.rename({"Total_Stops_num":"Total_Stops", "Duration_hours":"Duration"}, inplace=True)

X_test.head()

,Airline,Source,Destination,Additional_Info,Day_of_Journey,Total_Stops_num,Duration_hours
0,Jet Airways,Delhi,Cochin,No info,3,3,10
1,IndiGo,Kolkata,Banglore,No info,6,3,4
2,Jet Airways,Delhi,Cochin,In-flight meal not included,1,3,23
3,Multiple carriers,Delhi,Cochin,No info,1,3,13
4,Air Asia,Banglore,Delhi,No info,0,2,2


In [28]:
encoder = LabelEncoder()

In [29]:
for col in ['Airline', 'Source', 'Destination', 'Additional_Info']:
    encoder.fit(combined_dataframe[col])
    X[col] = encoder.transform(X[col])
    X_test[col] = encoder.transform(X_test[col])

In [30]:
X.head()

,Airline,Source,Destination,Additional_Info,Day_of_Journey,Total_Stops_num,Duration_hours
0,3,0,5,8,6,2,2
1,1,3,0,8,2,4,7
2,4,2,1,8,6,4,19
3,3,3,0,8,6,3,5
4,3,0,5,8,4,3,4


In [31]:
X_test.head()

,Airline,Source,Destination,Additional_Info,Day_of_Journey,Total_Stops_num,Duration_hours
0,4,2,1,8,3,3,10
1,3,3,0,8,6,3,4
2,4,2,1,5,1,3,23
3,6,2,1,8,1,3,13
4,0,0,2,8,0,2,2


In [32]:
# scaler = MinMaxScaler()

# X_train_scaled = pd.DataFrame(scaler.fit_transform(X))
# X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test))

# X_train_scaled.columns = X.columns.copy()
# X_test_scaled.columns = X_test.columns.copy()

In [34]:
param_grid = {"svr":{'kernel':['linear', 'rbf'], 'C':[0.1, 0.5, 1.0, 5.0, 10.0]},\
              "rf":{'n_estimators':[100,200,300,400], 'max_depth':[2]},\
             "xgb":{'n_estimators':[100,200,300,400,500], 'max_depth':[2,3], 'learning_rate':[0.05,0.1]}}

gridsearch = GridSearchCV(GradientBoostingRegressor(), param_grid=param_grid['xgb'], scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=2)

In [35]:
gridsearch.fit(X, y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_ite...one,
                        

In [36]:
y_pred = gridsearch.predict(X)

In [37]:
rmse = mean_squared_error(y, y_pred, squared=False)
print(rmse)

2148.6136775335217


In [38]:
y_pred = gridsearch.predict(X_test)

In [39]:
pd.DataFrame(y_pred, columns=['Price']).to_excel('data/Flight_Ticket_Participant_Datasets/Submission.xlsx', index=False)